# 🔍 Análise de Tráfego de Rede com PyShark

Este notebook carrega um arquivo `.pcap` gerado por `tcpdump` ou Wireshark, e faz uma análise das comunicações de rede, incluindo protocolos, IPs, DNS, HTTP e LDAP.

In [3]:
# ✅ Instalação necessária (caso esteja rodando localmente)
!pip install pyshark

In [4]:
import pyshark
from collections import Counter

# Caminho para o arquivo .pcap
pcap_path = '/home/jovyan/data/commnet.pcap'


#
import nest_asyncio
nest_asyncio.apply()


# Abre o arquivo .pcap
cap = pyshark.FileCapture(pcap_path, use_json=True)

protocols = Counter()
ips_src = Counter()
ips_dst = Counter()
dns_queries = []
ldap_auths = []
http_requests = []

for pkt in cap:
    try:
        if 'IP' in pkt:
            ips_src[pkt.ip.src] += 1
            ips_dst[pkt.ip.dst] += 1

        proto = pkt.highest_layer
        protocols[proto] += 1

        if 'DNS' in pkt:
            dns_queries.append(pkt.dns.qry_name)

        if 'HTTP' in pkt:
            if hasattr(pkt.http, 'request_full_uri'):
                http_requests.append(pkt.http.request_full_uri)

        if pkt.transport_layer == 'TCP' and pkt.tcp.dstport in ['389', '636']:
            ldap_auths.append(f"{pkt.ip.src} -> {pkt.ip.dst}")

    except AttributeError:
        continue

cap.close()

RuntimeError: This event loop is already running

In [ ]:
print("📡 Protocolos detectados:")
print(protocols)

print("\n🔎 Top 5 IPs de Origem:")
print(ips_src.most_common(5))

print("\n🎯 Top 5 IPs de Destino:")
print(ips_dst.most_common(5))

print("\n🌐 Consultas DNS:")
for q in dns_queries[:10]:
    print(f" - {q}")

print("\n🔐 Tentativas LDAP (porta 389 ou 636):")
for auth in ldap_auths[:10]:
    print(f" - {auth}")

print("\n📥 Requisições HTTP (exemplo):")
for req in http_requests[:5]:
    print(f" - {req}")